Burns paper v smith paper

In [7]:
import sys,glob
sys.path.append('/Users/simon/git/burns/code/')

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
data_dir = '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/'
blank_mz_tol = 5
mz_tol = 5

In [10]:
group1 = ['MSS1','MSS2','MSS3','MSS4','MSS5','MSS6','MSS13']
group2 = ['MSS10','MSS11','MSS14']
files1 = []
files2 = []
for g in group1:
    pattern = data_dir + g + 'a' + '*.mzML'
    matches = glob.glob(pattern)
    for match in matches:
        end_bit = match.split('/')[-1]
        bfile = end_bit.split('a')
        bfilename =  "".join(bfile[:-1]) + 'b' + bfile[-1]
        files1.append((match,data_dir+bfilename))
for g in group2:
    pattern = data_dir + g + 'a' + '*.mzML'
    matches = glob.glob(pattern)
    for match in matches:
        end_bit = match.split('/')[-1]
        bfile = end_bit.split('a')
        bfilename =  "".join(bfile[:-1]) + 'b' + bfile[-1]
        files2.append((match,data_dir + bfilename))

group1,paper1 = zip(*files1)
group2,paper2 = zip(*files2)

print len(group1),len(group2)
print group1
print group2
papers = paper1+paper2


blank_patterns = 'Blank_*'
blank_files = glob.glob(data_dir + blank_patterns)
print blank_files

13 9
('/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS1a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS2a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS3a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS4a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a1.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a2.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS5a3.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS6a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a1.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/simon_test/MSS13a2.mzML', '/Users/simon/Dropbox/BioR

In [11]:
group1 = paper1
group2 = paper2

In [12]:
from load_di import raw_load

# s = load_di(group1[0])
import glob
data = {}
for f in files1+files2:
    out_file = '../csv_files/' + f[0].split('/')[-1].split('.')[0]+'.csv'
#     data[f[0]] = load_di(f[0],blank_file = f[1],out_file = out_file,normalise = None,mz_tol = blank_mz_tol)
    data[f[0]] = raw_load(f[0])
    data[f[1]] = raw_load(f[1])

for b in blank_files:
    out_file = '../csv_files/' + b.split('/')[-1].split('.')[0]+'.csv'
    data[b] = raw_load(b)


Do some of Greedy matching

Write out a csv file for processing

In [16]:
from matching import Greedy
import numpy as np
from scipy.stats import ttest_ind
# for log in ['log','nolog']:
for log in ['log']:
#     for missing in ['nan',1e-5]:
    for missing in [1e-5]:
#         for normalise in ['none','tus']:
        for normalise in ['tus']:
            options = {'log':log,'missing':missing,'normalise':normalise}
            n_tests = 0

            if options['normalise'] == 'tus':
                normalised_data = {}
                for f,spec in data.items():
                    mz_list,i_list = zip(*spec)
                    tot_intensity = sum(i_list)
                    new_intensity = [1000.0*i/tot_intensity for i in i_list]
                    new_spec = zip(mz_list,new_intensity)
                    normalised_data[f] = new_spec
            elif options['normalise'] == 'none':
                normalised_data = data
            
            g = Greedy()
            mp = g.process(normalised_data,mz_tol)
            
            
#             new_mp = []
#             # remove peaksets that only appear in blanks
#             for ps in mp:
#                 for p in ps.peaks:
#                     if p[2] in group1 or p[2] in group2:
#                         found = True
#                 if found:
#                     new_mp.append(ps)
            
#             mp = new_mp
            
            display_names = {}
            for g in group1+group2:
                display_names[g] = g.split('/')[-1]
            import csv
            all_names = list(group1+group2)
            res = []


            heads = ["mz"] + [display_names[a] for a in all_names] + ["n_blank"] + ["pval"] +["qval"]
            for ps in mp:
                row = [ps.mean_mz]
                g1 = []
                g2 = []
                for n in all_names:
                    peak = filter(lambda x: x[2] == n,ps.peaks)
                    if len(peak) == 0:
                        row.append("nan")
                        # take max intensity
                        peak = sorted(peak,key = lambda x: x[1], reverse = True)
                    else:
                        row.append(peak[0][1])
                    if n in group1:
                        g1.append(row[-1])
                    if n in group2:
                        g2.append(row[-1])
                        
                ng1 = sum([1 for a in g1 if not a == "nan"])
                ng2 = sum([1 for a in g2 if not a == "nan"])
                if options['missing'] == 'nan':
                    g1 = [a for a in g1 if not a == "nan"]
                    g2 = [a for a in g2 if not a == "nan"]
                else:
                    g1 = [a if not a == "nan" else options["missing"] for a in g1]
                    g2 = [a if not a == "nan" else options["missing"] for a in g2]
                if options['log'] == 'log':
                    g1 = [np.log10(a) for a in g1]
                    g2 = [np.log10(a) for a in g2]
                
                
                
                if ng1 > 1 or ng2 > 1:
                    t,p = ttest_ind(g1,g2,equal_var=True)
                    n_tests += 1
                else:
                    p = 999
                    
                b_count = 0
                for n in blank_files:
                    peak = filter(lambda x: x[2] == n,ps.peaks)
                    if len(peak) > 0:
                        b_count += 1
                
                row.append(b_count)
                
                row.append(p)
                res.append(row)
            
            res = sorted(res,key = lambda x: x[-1])
            # compute q-value
            for i,r in enumerate(res):
                if r[-1] < 999:
                    q = r[-1]*n_tests / (1.0*(i+1))
                    r.append(q)
                else:
                    r.append("nan")
                    
            

            with open("spreadsheets/paper_v_paper_raw/paper_v_paper_raw_{}_{}_{}.csv".format(options['log'],options['missing'],options['normalise']),'w') as f:
                writer = csv.writer(f)
                writer.writerow(heads)
                fres = filter(lambda x: x[-1]<=0.05,res)
                for r in fres:
                    writer.writerow(r)

921974 peaks


In [14]:
print len(blank_files)

13
